In [7]:
from transformers import (
    AutoModelForCausalLM, AutoProcessor, 
    GenerationConfig, BitsAndBytesConfig
    )
from langchain.llms.base import LLM
from typing import List, Optional
from PIL import Image
import requests
import torch

In [8]:

class MolmoLLM(LLM):
    def __init__(self, model_name: str):
        # load the processor
        self.processor = AutoProcessor.from_pretrained(
            model_name,
            trust_remote_code=True, # 원격에서 커스텀 코드를 가져와 실행
            torch_dtype='auto',
            # torch_dtype=torch.float16,
            device_map='auto'
        )
        
        # 4비트 양자화 설정
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,   # 4비트로 로드
            bnb_4bit_use_double_quant=True,  # 더블 양자화 사용
            bnb_4bit_quant_type="nf4",  # 양자화 유형 선택 (nf4 또는 fp4)
            bnb_4bit_compute_dtype=torch.float16  # 계산을 위한 dtype (float16을 자주 사용)
        )
        
        # load the model
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True, # 원격에서 커스텀 코드를 가져와 실행
            torch_dtype='auto',
            quantization_config=bnb_config,  # 양자화 설정 전달
            device_map='auto', 
        )

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # 이미지 불러오기
        image_url = "https://picsum.photos/id/237/536/354"
        image = Image.open(requests.get(image_url, stream=True).raw)

        # 이미지와 텍스트를 Molmo 모델에 맞게 처리
        inputs = self.processor.process(
            images=[image],
            text=prompt
        )
        
        # 디바이스 이동 및 배치 크기 설정
        inputs = {k: v.to(self.model.device).unsqueeze(0) for k, v in inputs.items()}

        # 생성 옵션 설정
        generation_config = GenerationConfig(max_new_tokens=200, stop_strings=["<|endoftext|>"])

        # Molmo 모델로 텍스트 생성
        output = self.model.generate_from_batch(
            inputs,
            generation_config,
            tokenizer=self.processor.tokenizer
        )

        # 생성된 텍스트를 디코딩
        generated_tokens = output[0, inputs['input_ids'].size(1):]
        generated_text = self.processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)

        return generated_text

    @property
    def _identifying_params(self):
        return {"model_name": "Molmo-7B-D-0924"}

    @property
    def _llm_type(self):
        return "MolmoLLM"

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 프롬프트 템플릿 설정
template = """
Please describe the content of the image based on the following prompt: {prompt}
"""
prompt = PromptTemplate(
    input_variables=["prompt"],
    template=template
)

# Molmo LLM 생성
llm = MolmoLLM(model_name='allenai/Molmo-7B-D-0924')

# LLMChain 생성
llm_chain = LLMChain(llm=llm, prompt=prompt)

# 프롬프트 실행
result = llm_chain.run(prompt="Describe this image.")
print(result)


In [2]:
text = "이미지 속 새끼 고양이는 독특한 타비 무늬가 있는 아름다운 주황색과 흰색 털을 가지고 있습니다. 각 부분의 털 색상과 패턴을 분석해 보겠습니다.\n\n다리:\n다리는 주로 주황색이고 길이를 따라 더 어두운 주황색 줄무늬가 있습니다. 이것은 나머지 몸의 흰색 털과 뚜렷한 대조를 이룹니다.\n\n얼굴:\n새끼 고양이의 얼굴은 주로 흰색이고 코와 입은 분홍색입니다. 얼굴 중앙을 따라 이마에서 코까지 이어지는 주황색 줄무늬가 있습니다. 뺨은 흰색이고 눈 주변은 주황색으로 타비 고양이의 전형적인 가면과 같은 패턴을 만듭니다.\n\n등:\n새끼 고양이의 등은 더 어두운 주황색 줄무늬가 있는 주황색 털로 덮여 있습니다. 이 패턴은 타비 고양이의 특징이며 털에 깊이와 질감을 더합니다.\n\n배:\n가슴과 배 부분은 흰색으로 나머지 몸의 주황색 털과 뚜렷한 대조를 이룹니다. 이 흰색 색상은 발까지 확장되어"
print(text)

이미지 속 새끼 고양이는 독특한 타비 무늬가 있는 아름다운 주황색과 흰색 털을 가지고 있습니다. 각 부분의 털 색상과 패턴을 분석해 보겠습니다.

다리:
다리는 주로 주황색이고 길이를 따라 더 어두운 주황색 줄무늬가 있습니다. 이것은 나머지 몸의 흰색 털과 뚜렷한 대조를 이룹니다.

얼굴:
새끼 고양이의 얼굴은 주로 흰색이고 코와 입은 분홍색입니다. 얼굴 중앙을 따라 이마에서 코까지 이어지는 주황색 줄무늬가 있습니다. 뺨은 흰색이고 눈 주변은 주황색으로 타비 고양이의 전형적인 가면과 같은 패턴을 만듭니다.

등:
새끼 고양이의 등은 더 어두운 주황색 줄무늬가 있는 주황색 털로 덮여 있습니다. 이 패턴은 타비 고양이의 특징이며 털에 깊이와 질감을 더합니다.

배:
가슴과 배 부분은 흰색으로 나머지 몸의 주황색 털과 뚜렷한 대조를 이룹니다. 이 흰색 색상은 발까지 확장되어


In [ ]:
import re

text = "안녕! 😊 왼쪽 고양이는 '놀자!'라고 말하고 있어요. 🐱<br><br><color=#1A248D><b>#내새끼</b></color>"
# 이모티콘 제거 (유니코드 범위) + <> 예외 처리
cleaned_text = re.sub(r'[^\w\s.,!?\'<>]+', '', text)

print(cleaned_text)
# 출력: 안녕! 왼쪽 고양이는 '놀자!'라고 말하고 있어요.<br><br><color=#1A248D><b>#내The text `새끼</b></color>` in the cleaned_text variable is a part of the original text that was not removed by the regular expression pattern used to clean the text. The regular expression pattern `[^\w\s.,!?\'<>]+` is designed to remove characters that are not letters, digits, whitespace, periods, commas, exclamation marks, question marks, single quotes, or angle brackets.


안녕!  왼쪽 고양이는 '놀자!'라고 말하고 있어요. <br><br><color1A248D><b>내새끼<b><color>


In [ ]:
import requests

URL = 'http://221.163.19.142:55508/chatbot/chat?user=01'

# 전송할 이미지 파일 경로
image_path = './test_image.png'

# 이미지 파일 열기
with open(image_path, 'rb') as image_file:
    # multipart/form-data 형식으로 'image' 필드로 파일 전송
    files = {'image': ('다운로드.jpg', image_file, 'image/jpeg')}
    
    # POST 요청
    response = requests.post(URL, files=files)

# 응답 출력
print(response.status_code)
print(response.json())  # JSON 형식의 응답을 받는다면

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [1]:
import requests

URL = 'http://221.163.19.142:55508/chatbot/chat?user=01'
image_path = './test_image.png'

with open(image_path, 'rb') as image_file:
    files = {'image': ('다운로드.jpg', image_file, 'image/jpeg')}
    
    # 타임아웃 설정 (예: 10초)
    try:
        response = requests.post(URL, files=files, timeout=40)
        print(response.status_code)
        print(response.json())  # JSON 형식의 응답을 받는다면
    except requests.exceptions.Timeout:
        print("요청 시간 초과!")
    except requests.exceptions.RequestException as e:
        print(f"요청 중 에러 발생: {e}")


200
이 귀여운 오렌지색 새끼 고양이는 소파에 앉아 스마트폰을 들고 무언가에 푹 빠져 있는 모습이 정말 사랑스러워요. 호기심 가득한 눈빛으로 스크린을 바라보며 신기한 세상을 탐험 중인 것 같네요. 이 모습은 마치 작은 인간처럼 보이기도 해요! <br><br><color=#1A248D><b>#호기심왕</b></color> <color=#1A248D><b>#사랑스러운고양이</b></color> <color=#1A248D><b>#스마트폰의세계</b></color>


In [205]:
# 이미지를 넣으면 닮은 꼴의 JSON으로 인덱스를 추출함
# input : image 
# output : {index : integer}
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import (
    PromptTemplate, BasePromptTemplate, ChatPromptTemplate, 
    HumanMessagePromptTemplate, SystemMessagePromptTemplate
    )
import requests

text = "Explain the species and fur color of the animal based on the primary color of each part."
URL = f'http://221.163.19.142:55508/chatbot/chat_testing_prompt?text={text}'
image_path = './도토리2.jpg'

with open(image_path, 'rb') as image_file:
    files = {'image': ('다운로드.jpg', image_file, 'image/jpeg')}
    try:
        response = requests.post(URL, files=files, timeout=40)
        print(response.status_code)
        print(response.json())  # JSON 형식의 응답을 받는다면
    except requests.exceptions.Timeout:
        print("요청 시간 초과!")
    except requests.exceptions.RequestException as e:
        print(f"요청 중 에러 발생: {e}")



200
 The animal in the image is a dog with predominantly white fur. Its body is covered in short, white fur, giving it a clean and bright appearance. The dog's face features a mix of colors, with a white muzzle and a black nose. There's a noticeable brown patch on top of its head, adding a touch of contrast to its otherwise white coat. The dog's ears are also white, blending seamlessly with its body fur. Overall, the primary color of the dog's fur is white, with some brown accents on its head, creating a striking and adorable appearance.


In [206]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature=0
)


# 자료구조 정의 (pydantic)
class CusineRecipe(BaseModel):
    breed : str = Field(description="describe animal species")
    index : int = Field(description="""
                        matching the animal for predominantly color
                        1 == The main color is black,
                        2 == The main color is white,
                        3 == The main color is brown,
                        4 == It is mainly black, but other colors are available.
                        """)
    color : str = Field(description="what is main color")

# 출력 파서 정의
output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)



The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"breed": {"title": "Breed", "description": "describe animal species", "type": "string"}, "index": {"title": "Index", "description": "\n                        matching the animal for predominantly color\n                        1 == The main color is black,\n                        2 == The main color is white,\n                        3 == The main color is brown,\n                        4 == It is mainly black, but other colors are available.\n                        ", "type": "integer"}, "color": {"title": "Color", "descri

In [207]:
# prompt 구성
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

print(prompt)


input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"breed": {"title": "Breed", "description": "describe animal species", "type": "string"}, "index": {"title": "Index", "description": "\\n                        matching the animal for predominantly color\\n                        1 == The main color is black,\\n                        2 == The main color is white,\\n                        3 == The main color is brown,\\n                        4 == It is mainly black, but other colors ar

In [ ]:
chain = prompt | model | output_parser

val = chain.invoke({"query": response.json()})
val